## Implementacja populacji

#### o __init__ - konstruktor
- `chromoPrec` - dokładność reprezentacji chromosomów osobników populacji, liczba całkowita
- `size` - liczba osobnikow w populacji, liczba całkowita

####  o setPopulation - implementacja utworzenia populacji o rozmiarze `size`

#### o addIndividual - implementacja dodania osobnika do populacji
- `ind` - osobnik, obiekty klasy Individual

#### o bestSelection - implementacja selekcji najlepszych
- `percent` - procent najlepszych osobników, liczba całkowita
- `newPopulation` - najlepiej przystostowane osobniki z populacji, nowa populacja osobników

##### o calculateDistribution - implementacja wyznaczenia dystrybucji osobników dla danej populacji
- `temp` - dystrybucje osobników z populacji, lista
   
#### o rouletteWheelSelection - implementacja selekcji kołem ruletki
- `populationDist` - dystrybucja osobników populacji, lista
- `self.individuals[bestIndex]` - osobnik wybrany z populacji selekcją kołem ruletki, obiekt klasy Individual

In [1]:
class Population:
    def __init__(self, size, chromoPrec):
        self.size = size
        self.chromoPrec = chromoPrec
        if size > 0:
            self.individuals = self.setPopulation()
        else :
            self.individuals = []
        
    def setPopulation(self):    
        return [Individual(self.chromoPrec) for row in range(self.size)]
    
    def addIndividual(self, ind):    
        self.individuals.append(ind)
        self.size += 1
        
    def bestSelection(self, percent = 30):
        popAdaptation = []
        bestSelected = []
        N = int(self.size * percent / 100)

        for ind in self.individuals:
            popAdaptation.append(ind.boothFunc())  

        for i in range(0, N):  
            best = max(popAdaptation)
            bestSelected.append(best)
            popAdaptation.remove(best) 

        newPopulation = Population(0, 8)
        for best in bestSelected:
            for ind in self.individuals:
                if ind.boothFunc() == best:
                    newPopulation.addIndividual(ind)
        return newPopulation
        
    def calculateDistribution(self):
        temp = [] 
        # adaptation function
        for ind in self.individuals:
            temp.append(ind.reciprocalBoothFunc())
        adaptSum = sum(temp)
        # calculate every individual probability
        temp = [adaptation/adaptSum for adaptation in temp] 
        # calculate every individual probability
        for i in range(1, len(temp)):
            temp[i] += temp[i-1]
        return temp
    
    
    def rouletteWheelSelection(self, populationDist):
        # spin roulette wheel and find the best individual
        rand = random.random()
        previous = 0
        bestIndex = -1
        for index, value in enumerate(populationDist):
            if(previous <= rand <= value):
                bestIndex = index
            previous = value
        return self.individuals[bestIndex]

#### o tournamentSelection - implementacja selekcji turniejowej
- `population` - populacja, obiekt klasy Population
- `k` -  wielkość turnieju, liczba całkowita
- `pop.individuals[0]` - osobnik wybrany z populacji selekcją turniejową, obiekt klasy Individual

In [65]:
def tournamentSelection(population, k):
    pop = copy.deepcopy(population)
    popSize = len(pop.individuals)

    if popSize == 1:
        return pop.individuals[0]
    else:
        bits = pop.individuals[0].X.bits
        best = Population(0, bits)
        # create tournaments using all individuals from population
        while(popSize > 0):
            popSize = len(pop.individuals)
            tournament = Population(0, bits)

            # select k random individuals from population to new tournament (without repetition)
            if popSize >= k:
                for i in range(k):
                    randInd = random.choice(pop.individuals)
                    tournament.addIndividual(randInd)
                    pop.individuals.remove(randInd)
            elif popSize > 1:
                tournament = copy.deepcopy(pop)    
                pop.individuals.clear() 
            else:
                break

            # select the best individuals tournament (stochastic tournament selection)
            best.addIndividual(random.choice(tournament.individuals)) 

        return tournamentSelection(best, k)

#### o showRouletteWheel - implementacja prezentacji koła ruletki dla populacji
   - `individuals` - lista osobników, lista obiektów typu Individual
   - `populationDist` - dystrybucje osobników, lista 

In [62]:
def showRouletteWheel(individuals, populationDist):
    stringPop = []
    for ind in individuals:
        stringPop.append(str(ind.X.binary) + str(ind.Y.binary))
    fig = go.Figure(data = [go.Pie(labels = stringPop, values = populationDist)])
    fig.update_layout(legend = dict(x = 0, y = -2))
    fig.show()

# Przykłady użycia

In [25]:
import copy
import random 
import plotly.graph_objects as go

from ipynb.fs.full.Chromosome import Chromosome
from ipynb.fs.full.Individual import Individual

In [26]:
population = Population(10, 8)

In [27]:
for ind in population.individuals:
    print(ind.X.binary, ind.Y.binary)

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0] [0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1] [0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1] [1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1]
[0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0] [1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0] [0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
[1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,

In [28]:
bestPopulation = population.bestSelection()
for ind in bestPopulation.individuals:
    print(ind.X.binary, ind.Y.binary)

[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1] [1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1]
[1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1] [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1]
[1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1] [1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1]


In [29]:
dist = population.calculateDistribution()
bestRoulette = population.rouletteWheelSelection(dist)
print(bestRoulette.X.binary, bestRoulette.Y.binary)

[0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0] [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1]


In [63]:
showRouletteWheel(population.individuals, dist)

In [66]:
bestTournament = tournamentSelection(population, 3)
print(bestTournament.X.binary, bestTournament.Y.binary)

[1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1] [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1]
